# 1.Drive y carga Subset

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
!ls /content/drive/MyDrive


Mounted at /content/drive/
 borrador_fernando_180624.ipynb  'Colab Notebooks'   P101   vida_laboral.pdf


In [ ]:
import pandas as pd
df_subset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/df_subset.csv', keep_default_na=False)

In [ ]:
df_subset.shape

(28388, 3)

In [ ]:
df_subset.head()

,overall,reviewText,sentiment
0,5.0,Nice big bag. Will keep clothes and devices dry!,1
1,2.0,CHeap. Light. Small. Long enough for bike rack...,0
2,5.0,"What can you say...they're ""bean"" bags... Corn...",1
3,5.0,"It's a pillow, and an animal. Outside of it be...",1
4,5.0,What little girl doesn't love pink tennis ball...,1


In [ ]:
df_subset['sentiment'].value_counts()

,count
sentiment,
1,24159
0,4229


# 2. Preprocesado

En el dataset vamos a hacer las siguientes transformaciones:
  * Codificar a código ASCII para eliminar carcateres extraños y signos de puntuación
  * Eliminar stop words
  * Lematización
  * texto a minúsculas
  * Eliminar token < 3 caracteres

In [ ]:
# Instalar librerias y modelos
!pip install -U spacy download en_core_web_sm

In [ ]:
# Importar librerias
import pandas as pd
import spacy
import unicodedata
from bs4 import BeautifulSoup
import re

# Cargar modelo en ingles pequeño
nlp = spacy.load('en_core_web_sm')

# Normalización Unicode para eliminar caracteres extraños
def unicode_normalize(text):
  return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

def token_out(token):
  return token.is_punct or token.is_digit or token.is_stop or re.match(r"[^a-zA-Z0-9]", token.text) or len(token)<3

# Función de normalización
def normalize_text(text):
    text = unicode_normalize(text)
    text = BeautifulSoup(text, "html5lib").get_text()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Procesar el texto con Spacy
    doc = nlp(text)
    # Normalización de tokens
    normalized_tokens = []
    for token in doc:
        # Comprobar si es un signo de puntuación o digito o stop word
        if token_out(token):
            # Continuar
            continue

        # Tratar el resto de casos
        else:
            # lametizamos y convertir en minusculas
            normalized_tokens.append(token.lemma_.lower())


    # Unir tokens normalizados
    normalized_text = " ".join(normalized_tokens)
    return normalized_text


In [ ]:
# Preprocesado columna 'reviewText' del dataset
df_subset['reviewText'] = df_subset['reviewText'].apply(normalize_text)
df_subset.head()

,overall,reviewText,sentiment
0,5.0,nice big bag clothe device dry,1
1,2.0,cheap light small long bike rack coil snuggly ...,0
2,5.0,bean bag cornhole bag case strong sturdy updat...,1
3,5.0,pillow animal outside soft idea get colt beat ...,1
4,5.0,little girl doesn love pink tennis ball please...,1


In [ ]:
#  columna sentimentt del dataset
df_subset['sentiment'] = df_subset['overall'].apply(lambda x: 1 if x >= 4 else 0)
df_subset.head()

,overall,reviewText,sentiment
0,5.0,nice big bag clothe device dry,1
1,2.0,cheap light small long bike rack coil snuggly ...,0
2,5.0,bean bag cornhole bag case strong sturdy updat...,1
3,5.0,pillow animal outside soft idea get colt beat ...,1
4,5.0,little girl doesn love pink tennis ball please...,1


In [ ]:
df_subset['sentiment'].value_counts()

,count
sentiment,
1,24159
0,4229


In [ ]:
# Guardo el subset preprocesado
df_subset.to_csv('/content/drive/MyDrive/Colab Notebooks/subset_preprocessed.csv')